In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
!pip install sentence-transformers

In [ ]:
import tensorflow as tf
from tqdm import tqdm
from transformers import AutoTokenizer

In [ ]:
import pandas as pd
data_train=pd.read_csv("/content/drive/MyDrive/INPUT_knn/adaptive/train_data_adaptive_wiki.csv")
data_dev=pd.read_csv("/content/drive/MyDrive/INPUT_knn/adaptive/dev_data_adaptive_wiki.csv")
data_test=pd.read_csv("/content/drive/MyDrive/INPUT_knn/adaptive/test_data_adaptive_wiki.csv")

In [ ]:
claim_ls_train=data_train['claim'].tolist()
label_ls_train=data_train['k_val'].tolist()
claim_ls_dev=data_dev['claim'].tolist()
label_ls_dev=data_dev['k_val'].tolist()
claim_ls_test=data_test['claim'].tolist()
label_ls_test=data_test['k_val'].tolist()

In [ ]:
def updated_list(data):
  label_ts=[]
  for i in tqdm(data):
    label_ts.append(i-1)
  return label_ts

In [ ]:
label_ls_train_updated=updated_list(label_ls_train)
label_ls_dev_updated=updated_list(label_ls_dev)
label_ls_test_updated=updated_list(label_ls_test)

100%|██████████| 13089/13089 [00:00<00:00, 1165811.83it/s]


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-cased')

#Sentences are encoded by calling model.encode()
embeddings = model.encode(claim_ls_train)

#Print the embeddings
for sentence, embedding in zip(claim_ls_train, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    break

In [ ]:
from transformers import TFBertModel

In [ ]:
# mlm_model = TFBertModel.from_pretrained('bert-base-cased')

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# training_encodings = tokenizer(claim_ls_train,truncation=True,max_length=512,padding='max_length')
training_output=tf.keras.utils.to_categorical(label_ls_train_updated,num_classes=6)
# validation_encodings = tokenizer(claim_ls_dev,truncation=True,max_length=512,padding='max_length')
# validation_output=tf.keras.utils.to_categorical(label_ls_dev_updated,num_classes=6)
# test_encodings = tokenizer(claim_ls_test,truncation=True,max_length=512,padding='max_length')
# test_output=tf.keras.utils.to_categorical(label_ls_test_updated,num_classes=6)

In [ ]:
def get_ragged_tensor_representation(encodings):
  encodings['input_ids']=tf.dtypes.cast(tf.constant(encodings['input_ids']),tf.int32)
  encodings['token_type_ids']=tf.dtypes.cast(tf.constant(encodings['token_type_ids']),tf.int32)
  encodings['attention_mask']=tf.dtypes.cast(tf.constant(encodings['attention_mask']),tf.int32)
  return encodings

In [ ]:
def dense_tensor_creation(data,y):
  return data,y

In [ ]:
# training_input_bert=get_ragged_tensor_representation(training_encodings)
# validation_input_bert=get_ragged_tensor_representation(validation_encodings)
# test_input_bert=get_ragged_tensor_representation(test_encodings)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((embeddings,training_output))
# valid_dataset = tf.data.Dataset.from_tensor_slices((validation_input_bert,validation_output))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_input_bert,test_output))

In [ ]:
training_output.shape

(92062, 6)

In [ ]:
def make_batches(ds,batch_size):
  return (
      ds
      .cache()
      .map(dense_tensor_creation, num_parallel_calls=tf.data.AUTOTUNE)
      .shuffle(1000)
      .repeat()
      .batch(batch_size)
      .prefetch(tf.data.AUTOTUNE))

In [ ]:
def get_model():
  input_word_ids = tf.keras.layers.Input(shape=(768,), dtype=tf.int32,name="token_type_ids")
  # attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,name="attention_mask")

  # mlm_model.layers[0].trainable=False
  # sequence_output=mlm_model([input_word_ids,token_type_ids,attention_mask])
  out1=tf.keras.layers.Dense(512,activation="relu")(input_word_ids)
  out2=tf.keras.layers.Dense(256,activation="relu")(out1)
  out3=tf.keras.layers.Dense(128,activation="relu")(out2)
  out4=tf.keras.layers.Dense(64,activation="relu")(out3)
  out5=tf.keras.layers.Dense(6,activation="softmax")(out4)

  model=tf.keras.Model(inputs=[input_word_ids],outputs=out5)

  return model

In [ ]:
model=get_model()
optimizer = tf.keras.optimizers.Adam()
training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
training_accuracy = tf.keras.metrics.CategoricalAccuracy('training_accuracy', dtype=tf.float32)

In [ ]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy')

In [ ]:
data_trn=make_batches(train_dataset,64)
# data_valid=make_batches(valid_dataset,32)
# data_test=make_batches(test_dataset,32)

## Checking Part

In [ ]:
checkpoint_path="/content/"

In [ ]:
ckpt = tf.train.Checkpoint(model=model, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored; Model was trained for {} steps.'.format(ckpt.optimizer.iterations.numpy()))
else:
  print('Training from scratch!')

In [ ]:
@tf.function
def step_fn(inputs):
  """The computation to run on each TPU device."""
  images, labels = inputs
  with tf.GradientTape() as tape:
    logits = model(images, training=True)
    loss = tf.keras.losses.categorical_crossentropy(
        labels, logits)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
  training_loss.update_state(loss * tpu_strategy.num_replicas_in_sync)
  training_accuracy.update_state(labels, logits)


# @tf.function
# def valid_step(iterator):
#   """The step function for one training step"""

#   def step_fn(inputs):
#     """The computation to run on each TPU device."""
#     images, labels = inputs
#     logits = model(images, training=False)
#     loss = tf.keras.losses.categorical_crossentropy(labels, logits)
#     loss = tf.nn.compute_average_loss(loss, global_batch_size=global_batch_size)
#     validation_loss.update_state(loss * tpu_strategy.num_replicas_in_sync)
#     validation_accuracy.update_state(labels, logits)

#   tpu_strategy.run(step_fn, args=(next(iterator),))

## Training using model.fit

In [ ]:
train_iterator = iter(data_trn)

In [ ]:
steps=int(training_output.shape[0]/64)+1

In [ ]:
model.fit(train_iterator,steps_per_epoch=steps,epochs=50,class_weight=req_cls_wt)

Epoch 1/50
1439/1439 [==============================] - 11s 7ms/step - loss: 1.0518
Epoch 2/50
1439/1439 [==============================] - 8s 5ms/step - loss: 1.0423
Epoch 3/50
1439/1439 [==============================] - 7s 5ms/step - loss: 1.0411
Epoch 4/50
1439/1439 [==============================] - 8s 6ms/step - loss: 1.0399
Epoch 5/50
1439/1439 [==============================] - 7s 5ms/step - loss: 1.0384
Epoch 6/50
1439/1439 [==============================] - 8s 6ms/step - loss: 1.0365
Epoch 7/50
1439/1439 [==============================] - 7s 5ms/step - loss: 1.0354
Epoch 8/50
1439/1439 [==============================] - 8s 6ms/step - loss: 1.0340
Epoch 9/50
1439/1439 [==============================] - 6s 5ms/step - loss: 1.0325
Epoch 10/50
1439/1439 [==============================] - 8s 5ms/step - loss: 1.0297
Epoch 11/50
1439/1439 [==============================] - 7s 5ms/step - loss: 1.0289
Epoch 12/50
1439/1439 [==============================] - 7s 5ms/step - loss: 1.0272


In [ ]:
from collections import Counter
label_dict=Counter(label_ls_train_updated)

In [ ]:
req_cls_wt

{4: 2.4142713081492713,
 0: 1.0,
 5: 1.0,
 1: 1.0,
 3: 2.0281554888151776,
 2: 1.5300688908266167}

In [ ]:
label_dict

Counter({4: 1235, 0: 71567, 5: 8035, 1: 6418, 3: 1817, 2: 2990})

In [ ]:
import numpy as np
import math

# labels_dict : {ind_label: count_label}
# mu : parameter to tune

def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight


req_cls_wt=create_class_weight(label_dict)